In [21]:
import torch
import torch.utils.data as data

import numpy as np

In [25]:
def load_train_data(root):
    # Load train dataset for generating training data.
    with gzip.open(root, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
        data = data.reshape(-1, 28, 28)
    return data

In [4]:
import os
from os import walk
# return a tuple of arrays ([all directories inside input path], [all files inside input path])
def get_content_of_path(pathname, get_dir=True, get_filenames=True):
    f = []
    for (dirpath, dirnames, filenames) in walk(pathname):
        f.extend(filenames)
        break
    result = []
    if get_dir is False:
        return filenames
    if get_filenames is False:
        return dirnames
    return dirnames, filenames


def get_image_lists_from_dataset(datasets_path):
    train_bmode_list = []
    train_mask_list = []
    test_bmode_list = []
    test_mask_list = []

    if datasets_path != None:
        dataset_list = os.listdir(datasets_path)
        for dataset in dataset_list:
            bmode_list = [] # input
            mask_list = [] # output
            dataset_folder = datasets_path + dataset + "/"
            dataset_dirnames = get_content_of_path(dataset_folder, get_filenames=False)

            # loop through image files and put them in list corresponding to their filename
            for filename in dataset_dirnames:
                if filename != "bmode" and filename != "mask":
                    continue
                image_folder = dataset_folder + filename + "/"
                image_paths = get_content_of_path(image_folder, get_dir=False)
                image_paths = [image_folder + image_path for image_path in image_paths]
                if filename == "bmode":
                    bmode_list = image_paths
                elif filename == "mask":
                    mask_list = image_paths

            # assign bmode and mask list depending on dataset type
            if dataset == "train":
                train_bmode_list = bmode_list
                train_mask_list = mask_list
            elif dataset == "test":
                test_bmode_list = bmode_list
                test_mask_list = mask_list
    return train_bmode_list, train_mask_list, test_bmode_list, test_mask_list

In [5]:
datasets_path = "/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/"
train_bmode, train_mask, test_bmode, test_mask = get_image_lists_from_dataset(datasets_path)


In [6]:
from torchvision import transforms
from PIL import Image
# convert an array of image directory to an array of tensors
def img_to_tensor(img_dir_array):
    # tensor converter
    convert_tensor = transforms.ToTensor()
    # resulting tensor array
    img_tensor_array = []
    
    # start loop through img_dir_array
    for img_dir in img_dir_array:
        img = Image.open(img_dir)
        img_tensor_array.append(convert_tensor(img))
    return img_tensor_array

In [7]:
train_bmode_tensor = img_to_tensor(train_bmode)
train_mask_tensor = img_to_tensor(train_mask)

In [8]:
train_bmode_tensor1 = torch.Tensor(train_bmode_tensor)

ValueError: only one element tensors can be converted to Python scalars

In [11]:
import cv2
sample = cv2.imread(train_bmode[0])
train_bmode_img = [cv2.imread(img_dir) for img_dir in train_bmode]
train_mask_img = [cv2.imread(img_dir) for img_dir in train_mask]
test_bmode_img = [cv2.imread(img_dir) for img_dir in test_bmode]
train_mask_img = [cv2.imread(img_dir) for img_dir in test_mask]


In [23]:
np.array(train_bmode_img[0]).shape

(512, 512, 3)

In [ ]:
shape(train_bmode_tensor)

In [ ]:
data.TensorDataset(torch.Tensor(train_bmode_tensor), torch.Tensor(train_mask_tensor))

In [29]:
load_train_data("/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/bmode")

IsADirectoryError: [Errno 21] Is a directory: '/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/bmode'

In [32]:
import torch
from torchvision import datasets, transforms

In [39]:
transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()
                               ])
dataset = datasets.ImageFolder('/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

In [64]:
class UltrasoundSegmentation(data.Dataset):
    def __init__(self, folder_path):
        super(UltrasoundSegmentation, self).__init__()
        self.img_files = glob.glob(os.path.join(folder_path,'bmode','*.png'))
        self.mask_files = []
        for img_path in self.img_files:
             self.mask_files.append(os.path.join(folder_path,'mask',os.path.basename(img_path)))

    def __getitem__(self, index):
            img_path = self.img_files[index]
            mask_path = self.mask_files[index]
            data =  cv2.im_read(img_path)
            label = cv2.im_read(mask_path)
            return torch.from_numpy(data).float(), torch.from_numpy(label).float()

    def __len__(self):
        return len(self.img_files)

In [61]:
!pip install glob

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [62]:
import glob

In [65]:
train_folder = "/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/train/"
test_folder = "/gpfs/fs2/scratch/mdoyley_lab/GPU_VA/DL_dataset/test/"

train_set = UltrasoundSegmentation(train_folder)
test_set = UltrasoundSegmentation(test_folder)

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=1,
                 shuffle=False)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=1,
                shuffle=False)

In [66]:
print ("Number of training examples: " + str(train_set.__len__()))
print ("Number of testing examples: " + str(test_set.__len__()))

Number of training examples: 0
Number of testing examples: 0
